In [1]:
import sys
import os

# relative import error with python modules and what not, the path needs to change according to cwd :) 
project_root = os.path.abspath(os.path.join(os.getcwd(), "../../.."))
print(project_root)
if project_root not in sys.path:
    sys.path.append(project_root)

/home/turbouser44/research/Arxiv2Arxode


In [2]:
from datasets import load_from_disk

ds = load_from_disk("./datasets/")

In [3]:
unique_repos = set()
for subset in ds.values():
    unique_repos.update(subset['github'])
unique_repos, len(unique_repos)

({'https://github.com/IDEA-Research/Grounded-Segment-Anything',
  'https://github.com/NVIDIA/vid2vid',
  'https://github.com/Stability-AI/stablediffusion',
  'https://github.com/black0017/MedicalZooPytorch',
  'https://github.com/brightmart/text_classification',
  'https://github.com/deep-floyd/if',
  'https://github.com/dmlc/dgl',
  'https://github.com/eriklindernoren/PyTorch-GAN',
  'https://github.com/facebookresearch/esm',
  'https://github.com/google-research/bert',
  'https://github.com/huggingface/pytorch-image-models',
  'https://github.com/microsoft/muzic',
  'https://github.com/mlfoundations/open_clip',
  'https://github.com/salesforce/lavis',
  'https://github.com/tensorflow/tensor2tensor',
  'https://github.com/thuml/Time-Series-Library',
  'https://github.com/vinits5/learning3d',
  'https://github.com/xmu-xiaoma666/External-Attention-pytorch'},
 18)

In [4]:
import requests
import re

GITHUB_API = "https://api.github.com/repos/{}/readme"

ARXIV_REGEX = re.compile(r'arxiv.org/(abs|pdf)/\d{4}\.\d{5}')

def fetch_readme(repo_full_name, token=None):
    headers = {
        "X-GitHub-Api-Version": "2022-11-28"
    }
    if token:
        headers['Authorization'] = f'Bearer {token}'

    url = GITHUB_API.format(repo_full_name)
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        readme_data = response.json()
        readme_content = requests.get(readme_data['download_url']).text
        return readme_content
    else:
        print(f"Failed to fetch README for {repo_full_name}: {response.status_code}")
        return None

def check_for_arxiv_links(readme_content):
    m = ARXIV_REGEX.search(readme_content)
    if m:
        url = m.group(0)
        return url

def d(repo_list, token=None):
    xs = []
    for repo in repo_list:
        readme_content = fetch_readme(repo, token)
        if readme_content:
            x = check_for_arxiv_links(readme_content)
            if x:
                print(x)
                xs.append(x)
            else:
                print(None)
    return xs

github_token = os.environ.get("GH_PAT")

In [5]:
# "/".join(list(unique_repos)[0].split("/")[3:])

mlbench_repos = ["/".join(repo.split("/")[3:]) for repo in list(unique_repos)]
# mlbench_repos
arxiv_links = d(mlbench_repos, github_token)
# paperlist = d(["/".join(repo.split("/")[3:]) for repo in list(unique_repos)], github_token)


arxiv.org/pdf/2205.11487
arxiv.org/pdf/2003.00982
arxiv.org/abs/1610.09585
None
arxiv.org/pdf/2311.17049
arxiv.org/abs/2303.05499
arxiv.org/abs/1808.06601
arxiv.org/abs/1606.06650
arxiv.org/abs/2209.09019
arxiv.org/abs/2204.06125
arxiv.org/abs/2105.02358
arxiv.org/abs/2106.05630
arxiv.org/pdf/2010.09185
arxiv.org/abs/1607.01759
arxiv.org/abs/2407.13278
arxiv.org/abs/1908.08962
arxiv.org/abs/1812.02825
arxiv.org/abs/2212.07143


In [6]:
# arxiv_links[0].split("/")[-1]
arxiv_ids = [link.split("/")[-1] for link in arxiv_links]
arxiv_ids

['2205.11487',
 '2003.00982',
 '1610.09585',
 '2311.17049',
 '2303.05499',
 '1808.06601',
 '1606.06650',
 '2209.09019',
 '2204.06125',
 '2105.02358',
 '2106.05630',
 '2010.09185',
 '1607.01759',
 '2407.13278',
 '1908.08962',
 '1812.02825',
 '2212.07143']

In [14]:
from utils.arxiv import arxiv_search

# arxiv_search.ArxivScraper().download_papers(ids = arxiv_ids[:1], dirpath="./MLBench_papers/")
# arxiv_search.ArxivScraper().download_papers(ids = arxiv_ids[1:], dirpath="./MLBench_papers/")
for pid in arxiv_ids[2:]:
    arxiv_search.ArxivScraper().download_papers(ids = [pid], dirpath="./MLBench_papers/")
    print(f"Downloaded {pid}")

Downloaded 1610.09585
Downloaded 2311.17049
Downloaded 2303.05499
Downloaded 1808.06601
Downloaded 1606.06650
Downloaded 2209.09019
Downloaded 2204.06125
Downloaded 2105.02358
Downloaded 2106.05630
Downloaded 2010.09185
Downloaded 1607.01759
Downloaded 2407.13278
Downloaded 1908.08962
Downloaded 1812.02825
Downloaded 2212.07143


In [15]:
# ??re.search